<a href="https://colab.research.google.com/github/nurulsafitri321/HER_PBO/blob/main/library.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# API & Models


In [ ]:
# ============================================================================
# SECTION 1: API & DATA MODELS
# ============================================================================
# Jalankan cell ini terlebih dahulu untuk mendefinisikan struktur data

print("="*70)
print("SECTION 1: API & DATA MODELS")
print("="*70)

# Install dependencies
!pip install tabulate -q

from dataclasses import dataclass, field
from datetime import datetime
from typing import List, Optional

print("\n✓ Dependencies installed")

# ============================================================================
# DATA MODELS
# ============================================================================

@dataclass
class Book:
    """Model untuk data buku"""
    id: int
    title: str
    author: str
    category: str
    available: bool = True
    total_borrowed: int = 0

    def to_dict(self):
        return {
            'id': self.id,
            'title': self.title,
            'author': self.author,
            'category': self.category,
            'available': self.available,
            'total_borrowed': self.total_borrowed
        }


@dataclass
class User:
    """Model untuk data pengguna"""
    id: int
    name: str
    email: str
    borrowed_books: List[int] = field(default_factory=list)
    user_type: str = "mahasiswa"  # mahasiswa atau dosen

    def to_dict(self):
        return {
            'id': self.id,
            'name': self.name,
            'email': self.email,
            'borrowed_books': self.borrowed_books,
            'user_type': self.user_type
        }


@dataclass
class Transaction:
    """Model untuk data transaksi"""
    id: int
    user_id: int
    book_id: int
    borrow_date: datetime
    return_date: Optional[datetime] = None
    status: str = "borrowed"  # borrowed atau returned

    def to_dict(self):
        return {
            'id': self.id,
            'user_id': self.user_id,
            'book_id': self.book_id,
            'borrow_date': self.borrow_date.isoformat(),
            'return_date': self.return_date.isoformat() if self.return_date else None,
            'status': self.status
        }


print("✓ Data Models defined:")
print("  - Book")
print("  - User")
print("  - Transaction")

# ============================================================================
# DATABASE SCHEMA DOCUMENTATION
# ============================================================================

DATABASE_SCHEMA = """
DATABASE SCHEMA
===============

Table: Book
-----------
- id (int): Primary Key
- title (str): Judul buku
- author (str): Penulis buku
- category (str): Kategori buku
- available (bool): Status ketersediaan
- total_borrowed (int): Total kali dipinjam

Table: User
-----------
- id (int): Primary Key
- name (str): Nama pengguna
- email (str): Email pengguna
- borrowed_books (list): List ID buku yang dipinjam
- user_type (str): Tipe user (mahasiswa/dosen)

Table: Transaction
------------------
- id (int): Primary Key
- user_id (int): Foreign Key ke users
- book_id (int): Foreign Key ke books
- borrow_date (datetime): Tanggal peminjaman
- return_date (datetime): Tanggal pengembalian
- status (str): Status (borrowed/returned)
"""

print("\n" + DATABASE_SCHEMA)

print("\n" + "="*70)
print("✓ SECTION 1 COMPLETED - Lanjut ke Section 2")
print("="*70)

# Menu Pencarian


In [ ]:
# ============================================================================
# SECTION 2: MENU PENCARIAN BUKU/NOVEL
# ============================================================================
# Jalankan cell ini setelah Section 1
# Berisi: Modul pencarian dan fungsi-fungsi pencarian buku

print("="*70)
print("SECTION 2: MENU PENCARIAN BUKU/NOVEL")
print("="*70)

from tabulate import tabulate

# ============================================================================
# SEARCH MODULE - Modul Inti Pencarian
# ============================================================================

class SearchModule:
    """Modul untuk pencarian buku dengan berbagai kriteria"""

    @staticmethod
    def search_by_title(books: List[Book], keyword: str) -> List[Book]:
        """
        Mencari buku berdasarkan judul

        Args:
            books: List semua buku
            keyword: Kata kunci pencarian

        Returns:
            List buku yang judulnya mengandung keyword
        """
        return [book for book in books
                if keyword.lower() in book.title.lower()]

    @staticmethod
    def search_by_author(books: List[Book], keyword: str) -> List[Book]:
        """
        Mencari buku berdasarkan penulis

        Args:
            books: List semua buku
            keyword: Kata kunci pencarian

        Returns:
            List buku yang penulisnya mengandung keyword
        """
        return [book for book in books
                if keyword.lower() in book.author.lower()]

    @staticmethod
    def search_by_category(books: List[Book], category: str) -> List[Book]:
        """
        Mencari buku berdasarkan kategori

        Args:
            books: List semua buku
            category: Kategori yang dicari

        Returns:
            List buku dengan kategori tertentu
        """
        return [book for book in books
                if category.lower() == book.category.lower()]

    @staticmethod
    def search_available_books(books: List[Book]) -> List[Book]:
        """
        Mencari buku yang tersedia untuk dipinjam

        Args:
            books: List semua buku

        Returns:
            List buku yang tersedia
        """
        return [book for book in books if book.available]

    @staticmethod
    def search_borrowed_books(books: List[Book]) -> List[Book]:
        """
        Mencari buku yang sedang dipinjam

        Args:
            books: List semua buku

        Returns:
            List buku yang sedang dipinjam
        """
        return [book for book in books if not book.available]

    @staticmethod
    def advanced_search(books: List[Book],
                       title: str = None,
                       author: str = None,
                       category: str = None,
                       available_only: bool = False) -> List[Book]:
        """
        Pencarian advanced dengan multiple filter sekaligus

        Args:
            books: List semua buku
            title: Filter judul (optional)
            author: Filter penulis (optional)
            category: Filter kategori (optional)
            available_only: Hanya tampilkan buku tersedia

        Returns:
            List buku yang sesuai dengan semua filter
        """
        results = books.copy()

        if title:
            results = [b for b in results
                      if title.lower() in b.title.lower()]

        if author:
            results = [b for b in results
                      if author.lower() in b.author.lower()]

        if category:
            results = [b for b in results
                      if category.lower() == b.category.lower()]

        if available_only:
            results = [b for b in results if b.available]

        return results

    @staticmethod
    def get_categories(books: List[Book]) -> List[str]:
        """Mendapatkan list semua kategori yang ada"""
        categories = set(book.category for book in books)
        return sorted(list(categories))

    @staticmethod
    def get_books_by_popularity(books: List[Book], limit: int = 10) -> List[Book]:
        """Mendapatkan buku terpopuler berdasarkan total peminjaman"""
        sorted_books = sorted(books, key=lambda x: x.total_borrowed, reverse=True)
        return sorted_books[:limit]


print("✓ SearchModule berhasil didefinisikan")
print("  - search_by_title()")
print("  - search_by_author()")
print("  - search_by_category()")
print("  - search_available_books()")
print("  - search_borrowed_books()")
print("  - advanced_search()")
print("  - get_categories()")
print("  - get_books_by_popularity()")

# ============================================================================
# HELPER FUNCTIONS UNTUK DISPLAY
# ============================================================================

def display_books_table(books: List[Book], title: str = "DAFTAR BUKU"):
    """
    Menampilkan daftar buku dalam bentuk tabel yang rapi

    Args:
        books: List buku yang akan ditampilkan
        title: Judul tabel
    """
    if not books:
        print(f"\n{title}")
        print("="*70)
        print("Tidak ada buku yang ditemukan.")
        return

    print(f"\n{title}")
    print("="*70)

    table_data = [
        [
            b.id,
            b.title[:35] + "..." if len(b.title) > 35 else b.title,
            b.author[:25] + "..." if len(b.author) > 25 else b.author,
            b.category,
            "✓ Ya" if b.available else "✗ Tidak",
            b.total_borrowed
        ]
        for b in books
    ]

    print(tabulate(
        table_data,
        headers=["ID", "Judul Buku", "Penulis", "Kategori", "Tersedia", "Total Pinjam"],
        tablefmt="grid"
    ))
    print(f"\nTotal: {len(books)} buku")


def display_categories(books: List[Book]):
    """Menampilkan daftar kategori yang tersedia"""
    categories = SearchModule.get_categories(books)

    print("\n" + "="*70)
    print("KATEGORI BUKU YANG TERSEDIA")
    print("="*70)

    for i, cat in enumerate(categories, 1):
        count = len([b for b in books if b.category == cat])
        print(f"{i}. {cat} ({count} buku)")


print("✓ Helper functions berhasil didefinisikan")
print("  - display_books_table()")
print("  - display_categories()")

# ============================================================================
# TESTING SECTION 2
# ============================================================================

print("\n" + "="*70)
print("TESTING SEARCH MODULE")
print("="*70)

# Buat data dummy untuk testing
test_books_search = [
    Book(1, "Python Programming for Beginners", "John Smith", "Programming"),
    Book(2, "Advanced Python Techniques", "John Smith", "Programming"),
    Book(3, "Data Science with Python", "Jane Doe", "Data Science"),
    Book(4, "Machine Learning Basics", "Bob Wilson", "Data Science"),
    Book(5, "Web Development 101", "Alice Brown", "Web Development"),
    Book(6, "React.js Complete Guide", "Charlie Davis", "Web Development"),
    Book(7, "Database Design Fundamentals", "Eve Martinez", "Database"),
]

# Test beberapa buku sudah dipinjam
test_books_search[2].available = False
test_books_search[2].total_borrowed = 5
test_books_search[4].available = False
test_books_search[4].total_borrowed = 3

search_module = SearchModule()

print("\n[TEST 1] Mencari buku dengan judul 'Python'")
results = search_module.search_by_title(test_books_search, "Python")
display_books_table(results, "Hasil Pencarian: Judul 'Python'")

print("\n[TEST 2] Mencari buku dengan penulis 'John'")
results = search_module.search_by_author(test_books_search, "John")
display_books_table(results, "Hasil Pencarian: Penulis 'John'")

print("\n[TEST 3] Mencari buku kategori 'Data Science'")
results = search_module.search_by_category(test_books_search, "Data Science")
display_books_table(results, "Hasil Pencarian: Kategori 'Data Science'")

print("\n[TEST 4] Menampilkan buku yang tersedia")
results = search_module.search_available_books(test_books_search)
display_books_table(results, "Buku yang Tersedia untuk Dipinjam")

print("\n[TEST 5] Menampilkan buku yang sedang dipinjam")
results = search_module.search_borrowed_books(test_books_search)
display_books_table(results, "Buku yang Sedang Dipinjam")

print("\n[TEST 6] Advanced search: Programming + John Smith + Tersedia")
results = search_module.advanced_search(
    test_books_search,
    author="John",
    category="Programming",
    available_only=True
)
display_books_table(results, "Advanced Search Results")

print("\n[TEST 7] Menampilkan semua kategori")
display_categories(test_books_search)

print("\n[TEST 8] Buku terpopuler (Top 3)")
results = search_module.get_books_by_popularity(test_books_search, limit=3)
display_books_table(results, "Top 3 Buku Terpopuler")

print("\n" + "="*70)
print("✓✓✓ SECTION 2 COMPLETED ✓✓✓")
print("Semua fungsi pencarian berhasil ditest!")
print("Lanjut ke Section 3 - Menu Peminjaman Buku")
print("="*70)

# Menu Peminjaman


In [ ]:
# ============================================================================
# SECTION 3: MENU PEMINJAMAN BUKU
# ============================================================================
# File ini adalah bagian dari Sistem Perpustakaan Interaktif
# Bisa standalone atau terintegrasi dengan Section 1 & 2
# ============================================================================

print("\n" + "="*70)
print("SECTION 3: MENU PEMINJAMAN")
print("="*70)

from datetime import datetime
from typing import List, Dict, Optional
from tabulate import tabulate

# ============================================================================
# CEK DAN IMPORT DEPENDENCIES DARI SECTION 1
# ============================================================================

# Cek apakah models sudah ada dari Section 1
try:
    # Test apakah Book, User, Transaction sudah didefinisikan
    test_book = Book
    test_user = User
    test_transaction = Transaction
    print("✓ Models dari Section 1 ditemukan (Book, User, Transaction)")
    MODELS_IMPORTED = True
except NameError:
    print("⚠️  Models belum ada, akan menggunakan built-in models")
    MODELS_IMPORTED = False

    # Definisikan models jika belum ada
    from dataclasses import dataclass, field

    @dataclass
    class Book:
        id: int
        title: str
        author: str
        category: str
        available: bool = True
        total_borrowed: int = 0

    @dataclass
    class User:
        id: int
        name: str
        email: str
        borrowed_books: List[int] = field(default_factory=list)

    @dataclass
    class Transaction:
        id: int
        user_id: int
        book_id: int
        borrow_date: datetime
        return_date: Optional[datetime] = None
        status: str = "borrowed"

# ============================================================================
# BORROWING MODULE (Jika belum ada dari Section 1)
# ============================================================================

class BorrowingModule:
    """Modul untuk peminjaman buku"""

    def __init__(self):
        self.transactions: List[Transaction] = []
        self.transaction_counter = 1

    def borrow_book(self, user: User, book: Book) -> Dict[str, any]:
        """Meminjam buku"""
        if not book.available:
            return {"success": False, "message": "Buku tidak tersedia"}

        if len(user.borrowed_books) >= 3:
            return {"success": False, "message": "Maksimal peminjaman 3 buku"}

        # Update status
        book.available = False
        book.total_borrowed += 1
        user.borrowed_books.append(book.id)

        # Buat transaksi
        transaction = Transaction(
            id=self.transaction_counter,
            user_id=user.id,
            book_id=book.id,
            borrow_date=datetime.now()
        )
        self.transactions.append(transaction)
        self.transaction_counter += 1

        return {
            "success": True,
            "message": f"Buku '{book.title}' berhasil dipinjam",
            "transaction_id": transaction.id
        }

    def return_book(self, user: User, book: Book) -> Dict[str, any]:
        """Mengembalikan buku"""
        if book.id not in user.borrowed_books:
            return {"success": False, "message": "Buku tidak dipinjam oleh user ini"}

        # Update status
        book.available = True
        user.borrowed_books.remove(book.id)

        # Update transaksi
        for transaction in self.transactions:
            if (transaction.user_id == user.id and
                transaction.book_id == book.id and
                transaction.status == "borrowed"):
                transaction.return_date = datetime.now()
                transaction.status = "returned"
                break

        return {
            "success": True,
            "message": f"Buku '{book.title}' berhasil dikembalikan"
        }

# ============================================================================
# LIBRARY SYSTEM (Minimal untuk Section 3)
# ============================================================================

class LibrarySystem:
    """Sistem Perpustakaan untuk Section 3"""

    def __init__(self):
        self.books: List[Book] = []
        self.users: List[User] = []
        self.borrow_module = BorrowingModule()
        self.book_counter = 1
        self.user_counter = 1

    def add_book(self, title: str, author: str, category: str) -> Book:
        """Menambah buku baru"""
        book = Book(self.book_counter, title, author, category)
        self.books.append(book)
        self.book_counter += 1
        return book

    def get_book_by_id(self, book_id: int) -> Optional[Book]:
        """Mendapatkan buku berdasarkan ID"""
        return next((b for b in self.books if b.id == book_id), None)

    def get_available_books(self) -> List[Book]:
        """Mendapatkan semua buku yang tersedia"""
        return [b for b in self.books if b.available]

    def add_user(self, name: str, email: str) -> User:
        """Menambah user baru"""
        user = User(self.user_counter, name, email)
        self.users.append(user)
        self.user_counter += 1
        return user

    def get_user_by_id(self, user_id: int) -> Optional[User]:
        """Mendapatkan user berdasarkan ID"""
        return next((u for u in self.users if u.id == user_id), None)

    def borrow_book(self, user_id: int, book_id: int) -> Dict:
        """Meminjam buku"""
        user = self.get_user_by_id(user_id)
        book = self.get_book_by_id(book_id)

        if not user:
            return {"success": False, "message": "User tidak ditemukan"}
        if not book:
            return {"success": False, "message": "Buku tidak ditemukan"}

        return self.borrow_module.borrow_book(user, book)

    def return_book(self, user_id: int, book_id: int) -> Dict:
        """Mengembalikan buku"""
        user = self.get_user_by_id(user_id)
        book = self.get_book_by_id(book_id)

        if not user:
            return {"success": False, "message": "User tidak ditemukan"}
        if not book:
            return {"success": False, "message": "Buku tidak ditemukan"}

        return self.borrow_module.return_book(user, book)

    def display_books_table(self, books: List[Book] = None):
        """Menampilkan daftar buku dalam bentuk tabel"""
        if books is None:
            books = self.books

        if not books:
            print("📭 Tidak ada buku.")
            return

        table_data = [
            [b.id, b.title[:30], b.author[:20], b.category,
             "✓" if b.available else "✗", b.total_borrowed]
            for b in books
        ]

        print(tabulate(table_data,
                      headers=["ID", "Judul", "Penulis", "Kategori",
                              "Tersedia", "Total Pinjam"],
                      tablefmt="grid"))

    def display_users_table(self):
        """Menampilkan daftar user dalam bentuk tabel"""
        if not self.users:
            print("📭 Tidak ada user.")
            return

        table_data = [
            [u.id, u.name, u.email[:25], len(u.borrowed_books)]
            for u in self.users
        ]

        print(tabulate(table_data,
                      headers=["ID", "Nama", "Email", "Buku Dipinjam"],
                      tablefmt="grid"))

    def display_user_borrowed_books(self, user_id: int):
        """Menampilkan buku yang dipinjam user"""
        user = self.get_user_by_id(user_id)
        if not user:
            print("❌ User tidak ditemukan")
            return

        if not user.borrowed_books:
            print(f"ℹ️  {user.name} belum meminjam buku")
            return

        borrowed_books = [self.get_book_by_id(book_id)
                         for book_id in user.borrowed_books]
        borrowed_books = [b for b in borrowed_books if b]

        print(f"\n📚 Buku yang dipinjam oleh {user.name}:")
        self.display_books_table(borrowed_books)

# ============================================================================
# INISIALISASI LIBRARY
# ============================================================================

def initialize_library():
    """Inisialisasi library dengan data sample"""
    lib = LibrarySystem()

    # Tambah buku sample
    books_data = [
        ("Python for Beginners", "John Smith", "Programming"),
        ("Advanced Python", "John Smith", "Programming"),
        ("Data Science 101", "Jane Doe", "Data Science"),
        ("Machine Learning", "Bob Wilson", "Data Science"),
        ("Web Development", "Alice Brown", "Web Dev"),
        ("React Guide", "Charlie Davis", "Web Dev"),
        ("Database Design", "Eve Martinez", "Database"),
        ("SQL Mastery", "Frank Lee", "Database"),
    ]

    for title, author, category in books_data:
        lib.add_book(title, author, category)

    # Tambah user sample
    users_data = [
        ("Ahmad Santoso", "ahmad@email.com"),
        ("Siti Rahayu", "siti@email.com"),
        ("Budi Hartono", "budi@email.com"),
    ]

    for name, email in users_data:
        lib.add_user(name, email)

    return lib

# Cek apakah library sudah ada dari section sebelumnya
if 'library' not in globals():
    print("⚠️  Variable 'library' tidak ditemukan")
    print("🔄 Membuat library baru dengan data sample...")
    library = initialize_library()
    print(f"✓ Library berhasil dibuat!")
    print(f"  📚 {len(library.books)} buku")
    print(f"  👥 {len(library.users)} user")
else:
    print("✓ Menggunakan library yang sudah ada")

# ============================================================================
# FUNGSI HELPER
# ============================================================================

def get_input(prompt, input_type=str):
    """Helper untuk mendapatkan input dengan validasi"""
    while True:
        try:
            value = input(prompt)
            if input_type == int:
                return int(value)
            return value
        except ValueError:
            print("❌ Input tidak valid! Silakan coba lagi.")

def display_transactions_table(lib, user_id=None):
    """Menampilkan riwayat transaksi"""
    transactions = lib.borrow_module.transactions

    if user_id:
        transactions = [t for t in transactions if t.user_id == user_id]

    if not transactions:
        print("📭 Tidak ada transaksi.")
        return

    table_data = []
    for t in transactions:
        user = lib.get_user_by_id(t.user_id)
        book = lib.get_book_by_id(t.book_id)

        borrow_date = t.borrow_date.strftime("%d-%m-%Y %H:%M")
        return_date = (t.return_date.strftime("%d-%m-%Y %H:%M")
                      if t.return_date else "-")

        status = "✓ Dikembalikan" if t.status == "returned" else "📖 Dipinjam"

        table_data.append([
            t.id,
            user.name if user else "?",
            book.title[:25] if book else "?",
            borrow_date,
            return_date,
            status
        ])

    print(tabulate(table_data,
                  headers=["ID", "User", "Buku", "Tgl Pinjam",
                          "Tgl Kembali", "Status"],
                  tablefmt="grid"))

# ============================================================================
# MENU FUNCTIONS
# ============================================================================

def menu_pinjam_buku(lib):
    """Menu untuk meminjam buku"""
    print("\n" + "="*70)
    print("📖 PINJAM BUKU")
    print("="*70)

    print("\n👥 Daftar User:")
    lib.display_users_table()

    user_id = get_input("\n➤ Masukkan ID User: ", int)
    user = lib.get_user_by_id(user_id)

    if not user:
        print("❌ User tidak ditemukan!")
        return

    current_borrowed = len(user.borrowed_books)
    print(f"\nℹ️  {user.name} telah meminjam {current_borrowed}/3 buku")

    if current_borrowed >= 3:
        print("⚠️  Maksimal peminjaman adalah 3 buku!")
        return

    print("\n📚 Buku yang Tersedia:")
    available = lib.get_available_books()

    if not available:
        print("❌ Tidak ada buku tersedia.")
        return

    lib.display_books_table(available)

    book_id = get_input("\n➤ ID Buku yang ingin dipinjam: ", int)

    print("\n⏳ Memproses...")
    result = lib.borrow_book(user_id, book_id)

    if result['success']:
        print(f"\n✅ BERHASIL!")
        print(f"   {result['message']}")
        print(f"   📋 ID Transaksi: {result.get('transaction_id')}")
        print(f"   📅 {datetime.now().strftime('%d-%m-%Y %H:%M')}")
        print(f"\n📊 {user.name} sekarang punya {len(user.borrowed_books)} buku")
    else:
        print(f"\n❌ GAGAL: {result['message']}")

def menu_kembalikan_buku(lib):
    """Menu untuk mengembalikan buku"""
    print("\n" + "="*70)
    print("📥 KEMBALIKAN BUKU")
    print("="*70)

    print("\n👥 Daftar User:")
    lib.display_users_table()

    user_id = get_input("\n➤ Masukkan ID User: ", int)
    user = lib.get_user_by_id(user_id)

    if not user:
        print("❌ User tidak ditemukan!")
        return

    if not user.borrowed_books:
        print(f"\nℹ️  {user.name} tidak punya buku yang perlu dikembalikan")
        return

    lib.display_user_borrowed_books(user_id)

    book_id = get_input("\n➤ ID Buku yang dikembalikan: ", int)

    print("\n⏳ Memproses...")
    result = lib.return_book(user_id, book_id)

    if result['success']:
        print(f"\n✅ BERHASIL!")
        print(f"   {result['message']}")
        print(f"   📅 {datetime.now().strftime('%d-%m-%Y %H:%M')}")
        print(f"\n📊 {user.name} sekarang punya {len(user.borrowed_books)} buku")
    else:
        print(f"\n❌ GAGAL: {result['message']}")

def menu_lihat_pinjaman(lib):
    """Menu melihat pinjaman user"""
    print("\n" + "="*70)
    print("📚 BUKU YANG DIPINJAM USER")
    print("="*70)

    print("\n👥 Daftar User:")
    lib.display_users_table()

    user_id = get_input("\n➤ ID User: ", int)
    lib.display_user_borrowed_books(user_id)

def menu_riwayat(lib):
    """Menu riwayat transaksi"""
    print("\n" + "="*70)
    print("📋 RIWAYAT TRANSAKSI")
    print("="*70)

    print("\n1. Semua Transaksi")
    print("2. Transaksi User Tertentu")

    choice = get_input("\n➤ Pilih (1-2): ", int)

    if choice == 1:
        print("\n📊 Semua Transaksi:")
        display_transactions_table(lib)
    elif choice == 2:
        print("\n👥 Daftar User:")
        lib.display_users_table()
        user_id = get_input("\n➤ ID User: ", int)
        print(f"\n📊 Transaksi User #{user_id}:")
        display_transactions_table(lib, user_id)

# ============================================================================
# MAIN MENU
# ============================================================================

def show_menu():
    """Tampilkan menu"""
    print("\n" + "="*70)
    print("📖 MENU PEMINJAMAN")
    print("="*70)
    print("\n8.  📖 Pinjam Buku")
    print("9.  📥 Kembalikan Buku")
    print("10. 📚 Lihat Pinjaman User")
    print("11. 📋 Riwayat Transaksi")
    print("0.  ◀️  Keluar")
    print("="*70)

def run_borrowing_menu(lib):
    """Jalankan menu peminjaman"""
    print("\n🚀 MENU PEMINJAMAN AKTIF")

    while True:
        show_menu()
        choice = get_input("\n➤ Pilih (0, 8-11): ", int)

        if choice == 0:
            print("\n👋 Keluar dari Menu Peminjaman")
            break
        elif choice == 8:
            menu_pinjam_buku(lib)
        elif choice == 9:
            menu_kembalikan_buku(lib)
        elif choice == 10:
            menu_lihat_pinjaman(lib)
        elif choice == 11:
            menu_riwayat(lib)
        else:
            print("❌ Pilihan tidak valid!")

        input("\n⏸️  Tekan ENTER...")

# ============================================================================
# AUTO RUN
# ============================================================================

print("\n" + "="*70)
print("✓ Section 3 siap digunakan!")
print("="*70)

# Auto run
print("\n🚀 Menjalankan Menu Peminjaman...\n")
run_borrowing_menu(library)


SECTION 3: MENU PEMINJAMAN
✓ Models dari Section 1 ditemukan (Book, User, Transaction)
⚠️  Variable 'library' tidak ditemukan
🔄 Membuat library baru dengan data sample...
✓ Library berhasil dibuat!
  📚 8 buku
  👥 3 user

✓ Section 3 siap digunakan!

🚀 Menjalankan Menu Peminjaman...


🚀 MENU PEMINJAMAN AKTIF

📖 MENU PEMINJAMAN

8.  📖 Pinjam Buku
9.  📥 Kembalikan Buku
10. 📚 Lihat Pinjaman User
11. 📋 Riwayat Transaksi
0.  ◀️  Keluar


# Menu Pengembalian


In [ ]:
# ============================================================================
# SECTION 4: MENU PENGEMBALIAN BUKU
# ============================================================================
# File ini adalah bagian dari Sistem Perpustakaan Interaktif
# Fokus pada fitur pengembalian buku dengan detail yang lengkap
# ============================================================================

print("\n" + "="*70)
print("SECTION 4: MENU PENGEMBALIAN")
print("="*70)

from datetime import datetime, timedelta
from typing import List, Dict, Optional
from tabulate import tabulate

# ============================================================================
# CEK DAN IMPORT DEPENDENCIES
# ============================================================================

# Cek apakah models sudah ada dari Section 1
try:
    test_book = Book
    test_user = User
    test_transaction = Transaction
    print("✓ Models dari Section 1 ditemukan")
    MODELS_IMPORTED = True
except NameError:
    print("⚠️  Models belum ada, akan menggunakan built-in models")
    MODELS_IMPORTED = False

    from dataclasses import dataclass, field

    @dataclass
    class Book:
        id: int
        title: str
        author: str
        category: str
        available: bool = True
        total_borrowed: int = 0

    @dataclass
    class User:
        id: int
        name: str
        email: str
        borrowed_books: List[int] = field(default_factory=list)

    @dataclass
    class Transaction:
        id: int
        user_id: int
        book_id: int
        borrow_date: datetime
        return_date: Optional[datetime] = None
        status: str = "borrowed"

# Cek apakah library sudah ada
if 'library' not in globals():
    print("❌ Error: Variable 'library' tidak ditemukan!")
    print("⚠️  Silakan jalankan Section 1 atau Section 3 terlebih dahulu")
    # Buat library minimal untuk testing
    class MinimalLibrary:
        def __init__(self):
            self.books = []
            self.users = []
            from types import SimpleNamespace
            self.borrow_module = SimpleNamespace(transactions=[])

        def get_user_by_id(self, user_id):
            return next((u for u in self.users if u.id == user_id), None)

        def get_book_by_id(self, book_id):
            return next((b for b in self.books if b.id == book_id), None)

    library = MinimalLibrary()
    print("⚠️  Menggunakan library minimal untuk testing")
else:
    print("✓ Menggunakan library yang sudah ada")

# ============================================================================
# FUNGSI HELPER
# ============================================================================

def get_input(prompt, input_type=str):
    """Helper untuk mendapatkan input dengan validasi"""
    while True:
        try:
            value = input(prompt)
            if input_type == int:
                return int(value)
            return value
        except ValueError:
            print("❌ Input tidak valid! Silakan coba lagi.")
        except KeyboardInterrupt:
            print("\n⚠️  Input dibatalkan")
            return None

def calculate_borrow_duration(borrow_date: datetime, return_date: datetime) -> Dict:
    """Menghitung durasi peminjaman"""
    duration = return_date - borrow_date
    days = duration.days
    hours = duration.seconds // 3600
    minutes = (duration.seconds % 3600) // 60

    return {
        "days": days,
        "hours": hours,
        "minutes": minutes,
        "total_hours": days * 24 + hours
    }

def check_late_return(borrow_date: datetime, return_date: datetime, max_days: int = 7) -> Dict:
    """Cek apakah pengembalian terlambat"""
    duration = calculate_borrow_duration(borrow_date, return_date)
    days = duration["days"]

    is_late = days > max_days
    late_days = max(0, days - max_days)

    # Hitung denda (contoh: Rp 1.000 per hari)
    fine = late_days * 1000

    return {
        "is_late": is_late,
        "late_days": late_days,
        "fine": fine,
        "max_days": max_days
    }

# ============================================================================
# DISPLAY FUNCTIONS
# ============================================================================

def display_users_with_borrowed_books(lib):
    """Menampilkan user yang sedang meminjam buku"""
    users_with_books = [u for u in lib.users if len(u.borrowed_books) > 0]

    if not users_with_books:
        print("📭 Tidak ada user yang sedang meminjam buku")
        return None

    table_data = []
    for u in users_with_books:
        table_data.append([
            u.id,
            u.name,
            u.email[:30],
            len(u.borrowed_books),
            "📚" * len(u.borrowed_books)
        ])

    print(tabulate(table_data,
                  headers=["ID", "Nama", "Email", "Jumlah", "Status"],
                  tablefmt="grid"))

    return users_with_books

def display_borrowed_books_detail(lib, user_id: int):
    """Menampilkan detail buku yang dipinjam dengan info waktu"""
    user = lib.get_user_by_id(user_id)

    if not user:
        print("❌ User tidak ditemukan")
        return None

    if not user.borrowed_books:
        print(f"ℹ️  {user.name} tidak memiliki buku yang perlu dikembalikan")
        return None

    # Ambil transaksi user yang masih aktif
    active_transactions = [
        t for t in lib.borrow_module.transactions
        if t.user_id == user_id and t.status == "borrowed"
    ]

    if not active_transactions:
        print("ℹ️  Tidak ada transaksi aktif")
        return None

    print(f"\n📚 Buku yang Dipinjam oleh: {user.name}")
    print("="*70)

    table_data = []
    for trans in active_transactions:
        book = lib.get_book_by_id(trans.book_id)
        if book:
            duration = calculate_borrow_duration(trans.borrow_date, datetime.now())
            late_info = check_late_return(trans.borrow_date, datetime.now())

            days_str = f"{duration['days']} hari"
            if duration['days'] == 0:
                days_str = f"{duration['hours']} jam"

            status = "✅ Normal" if not late_info['is_late'] else f"⚠️ Terlambat {late_info['late_days']} hari"

            table_data.append([
                book.id,
                book.title[:35],
                book.author[:20],
                trans.borrow_date.strftime("%d-%m-%Y"),
                days_str,
                status
            ])

    print(tabulate(table_data,
                  headers=["ID", "Judul", "Penulis", "Tgl Pinjam", "Durasi", "Status"],
                  tablefmt="grid"))

    return active_transactions

def display_return_summary(lib, user: User, book: Book, transaction: Transaction):
    """Menampilkan ringkasan pengembalian"""
    print("\n" + "="*70)
    print("📋 RINGKASAN PENGEMBALIAN")
    print("="*70)

    return_date = datetime.now()
    duration = calculate_borrow_duration(transaction.borrow_date, return_date)
    late_info = check_late_return(transaction.borrow_date, return_date)

    print(f"\n👤 Peminjam    : {user.name}")
    print(f"📚 Buku        : {book.title}")
    print(f"✍️  Penulis     : {book.author}")
    print(f"📅 Tgl Pinjam  : {transaction.borrow_date.strftime('%d-%m-%Y %H:%M')}")
    print(f"📅 Tgl Kembali : {return_date.strftime('%d-%m-%Y %H:%M')}")
    print(f"⏱️  Durasi      : {duration['days']} hari, {duration['hours']} jam, {duration['minutes']} menit")

    print(f"\n{'─'*70}")

    if late_info['is_late']:
        print(f"⚠️  STATUS      : TERLAMBAT")
        print(f"📆 Melebihi    : {late_info['late_days']} hari")
        print(f"💰 Denda       : Rp {late_info['fine']:,}")
        print(f"\n⚠️  Catatan: Batas peminjaman adalah {late_info['max_days']} hari")
    else:
        print(f"✅ STATUS      : TEPAT WAKTU")
        print(f"💚 Tidak ada denda")
        remaining = late_info['max_days'] - duration['days']
        if remaining > 0:
            print(f"📊 Sisa waktu  : {remaining} hari")

    print(f"{'─'*70}\n")

    return late_info

def display_return_history(lib, limit: int = 10):
    """Menampilkan riwayat pengembalian terakhir"""
    returned_transactions = [
        t for t in lib.borrow_module.transactions
        if t.status == "returned"
    ]

    if not returned_transactions:
        print("📭 Belum ada riwayat pengembalian")
        return

    # Sort by return date (newest first)
    returned_transactions.sort(key=lambda x: x.return_date, reverse=True)
    recent = returned_transactions[:limit]

    print(f"\n📜 {len(recent)} Pengembalian Terakhir:")
    print("="*70)

    table_data = []
    for trans in recent:
        user = lib.get_user_by_id(trans.user_id)
        book = lib.get_book_by_id(trans.book_id)

        if user and book:
            duration = calculate_borrow_duration(trans.borrow_date, trans.return_date)
            late_info = check_late_return(trans.borrow_date, trans.return_date)

            status = "✅" if not late_info['is_late'] else f"⚠️ +{late_info['late_days']}d"

            table_data.append([
                trans.id,
                user.name[:20],
                book.title[:30],
                trans.return_date.strftime("%d-%m-%Y"),
                f"{duration['days']}d",
                status
            ])

    print(tabulate(table_data,
                  headers=["ID", "User", "Buku", "Tgl Kembali", "Durasi", "Status"],
                  tablefmt="grid"))

def display_late_returns_report(lib):
    """Menampilkan laporan pengembalian terlambat"""
    all_returned = [
        t for t in lib.borrow_module.transactions
        if t.status == "returned"
    ]

    late_returns = []
    for trans in all_returned:
        late_info = check_late_return(trans.borrow_date, trans.return_date)
        if late_info['is_late']:
            late_returns.append((trans, late_info))

    if not late_returns:
        print("\n✅ Tidak ada pengembalian terlambat")
        return

    print(f"\n⚠️  LAPORAN PENGEMBALIAN TERLAMBAT")
    print(f"Total: {len(late_returns)} kasus")
    print("="*70)

    table_data = []
    total_fine = 0

    for trans, late_info in late_returns:
        user = lib.get_user_by_id(trans.user_id)
        book = lib.get_book_by_id(trans.book_id)

        if user and book:
            total_fine += late_info['fine']
            table_data.append([
                trans.id,
                user.name[:20],
                book.title[:25],
                trans.return_date.strftime("%d-%m-%Y"),
                late_info['late_days'],
                f"Rp {late_info['fine']:,}"
            ])

    print(tabulate(table_data,
                  headers=["ID", "User", "Buku", "Tgl Kembali", "Terlambat", "Denda"],
                  tablefmt="grid"))

    print(f"\n💰 Total Denda: Rp {total_fine:,}")

# ============================================================================
# MENU FUNCTIONS
# ============================================================================

def menu_kembalikan_buku(lib):
    """Menu utama untuk mengembalikan buku"""
    print("\n" + "="*70)
    print("📥 PENGEMBALIAN BUKU")
    print("="*70)

    # Tampilkan user yang sedang meminjam
    print("\n👥 User yang Sedang Meminjam Buku:")
    users = display_users_with_borrowed_books(lib)

    if not users:
        return

    user_id = get_input("\n➤ Masukkan ID User: ", int)
    if user_id is None:
        return

    user = lib.get_user_by_id(user_id)
    if not user:
        print("❌ User tidak ditemukan!")
        return

    # Tampilkan buku yang dipinjam dengan detail
    transactions = display_borrowed_books_detail(lib, user_id)

    if not transactions:
        return

    book_id = get_input("\n➤ Masukkan ID Buku yang akan dikembalikan: ", int)
    if book_id is None:
        return

    book = lib.get_book_by_id(book_id)
    if not book:
        print("❌ Buku tidak ditemukan!")
        return

    # Cari transaksi yang sesuai
    transaction = None
    for trans in transactions:
        if trans.book_id == book_id:
            transaction = trans
            break

    if not transaction:
        print("❌ Transaksi tidak ditemukan!")
        return

    # Tampilkan ringkasan sebelum konfirmasi
    late_info = display_return_summary(lib, user, book, transaction)

    # Konfirmasi
    confirm = input("➤ Proses pengembalian? (y/n): ").lower()

    if confirm != 'y':
        print("❌ Pengembalian dibatalkan")
        return

    # Proses pengembalian
    print("\n⏳ Memproses pengembalian...")
    result = lib.return_book(user_id, book_id)

    if result['success']:
        print(f"\n✅ PENGEMBALIAN BERHASIL!")
        print(f"   {result['message']}")

        if late_info['is_late']:
            print(f"\n💰 Denda yang harus dibayar: Rp {late_info['fine']:,}")
            print(f"   (Terlambat {late_info['late_days']} hari)")

        print(f"\n📊 Statistik {user.name}:")
        print(f"   • Buku tersisa: {len(user.borrowed_books)}")
        print(f"   • Slot tersedia: {3 - len(user.borrowed_books)}")
    else:
        print(f"\n❌ PENGEMBALIAN GAGAL!")
        print(f"   {result['message']}")

def menu_kembalikan_batch(lib):
    """Menu untuk mengembalikan banyak buku sekaligus"""
    print("\n" + "="*70)
    print("📥 PENGEMBALIAN BUKU (BATCH)")
    print("="*70)

    print("\n👥 User yang Sedang Meminjam Buku:")
    users = display_users_with_borrowed_books(lib)

    if not users:
        return

    user_id = get_input("\n➤ Masukkan ID User: ", int)
    if user_id is None:
        return

    user = lib.get_user_by_id(user_id)
    if not user:
        print("❌ User tidak ditemukan!")
        return

    transactions = display_borrowed_books_detail(lib, user_id)

    if not transactions:
        return

    print(f"\n📚 {user.name} meminjam {len(user.borrowed_books)} buku")
    print("Pilihan:")
    print("1. Kembalikan semua buku")
    print("2. Pilih buku yang akan dikembalikan")

    choice = get_input("\n➤ Pilih (1-2): ", int)

    if choice == 1:
        # Kembalikan semua
        confirm = input(f"➤ Kembalikan semua {len(user.borrowed_books)} buku? (y/n): ").lower()
        if confirm != 'y':
            print("❌ Dibatalkan")
            return

        book_ids = user.borrowed_books.copy()
    elif choice == 2:
        # Pilih buku
        print("\n➤ Masukkan ID buku yang akan dikembalikan (pisahkan dengan koma)")
        print("   Contoh: 1,3,5")
        ids_input = input("➤ ID Buku: ")
        try:
            book_ids = [int(x.strip()) for x in ids_input.split(',')]
        except:
            print("❌ Format tidak valid!")
            return
    else:
        print("❌ Pilihan tidak valid!")
        return

    # Proses pengembalian batch
    print(f"\n⏳ Memproses pengembalian {len(book_ids)} buku...")

    success_count = 0
    total_fine = 0

    for book_id in book_ids:
        book = lib.get_book_by_id(book_id)
        if not book:
            print(f"⚠️  Buku ID {book_id} tidak ditemukan, dilewati")
            continue

        # Cari transaksi
        trans = None
        for t in transactions:
            if t.book_id == book_id:
                trans = t
                break

        if not trans:
            print(f"⚠️  Transaksi untuk buku ID {book_id} tidak ditemukan, dilewati")
            continue

        # Cek denda
        late_info = check_late_return(trans.borrow_date, datetime.now())
        if late_info['is_late']:
            total_fine += late_info['fine']

        # Kembalikan
        result = lib.return_book(user_id, book_id)
        if result['success']:
            success_count += 1
            status = "✅" if not late_info['is_late'] else f"⚠️ Denda: Rp {late_info['fine']:,}"
            print(f"   {status} {book.title[:40]}")

    print(f"\n{'='*70}")
    print(f"✅ Berhasil mengembalikan {success_count}/{len(book_ids)} buku")

    if total_fine > 0:
        print(f"💰 Total denda: Rp {total_fine:,}")

    print(f"\n📊 {user.name} sekarang memiliki {len(user.borrowed_books)} buku")

def menu_riwayat_pengembalian(lib):
    """Menu untuk melihat riwayat pengembalian"""
    print("\n" + "="*70)
    print("📜 RIWAYAT PENGEMBALIAN")
    print("="*70)

    print("\n1. Riwayat Pengembalian Terakhir")
    print("2. Riwayat Pengembalian User Tertentu")
    print("3. Laporan Pengembalian Terlambat")

    choice = get_input("\n➤ Pilih (1-3): ", int)

    if choice == 1:
        display_return_history(lib, 10)
    elif choice == 2:
        print("\n👥 Daftar User:")
        lib.display_users_table()
        user_id = get_input("\n➤ ID User: ", int)

        user_returns = [
            t for t in lib.borrow_module.transactions
            if t.user_id == user_id and t.status == "returned"
        ]

        if not user_returns:
            print(f"📭 User belum pernah mengembalikan buku")
            return

        print(f"\n📊 Riwayat Pengembalian User #{user_id}:")
        user_returns.sort(key=lambda x: x.return_date, reverse=True)

        table_data = []
        for trans in user_returns:
            book = lib.get_book_by_id(trans.book_id)
            duration = calculate_borrow_duration(trans.borrow_date, trans.return_date)
            late_info = check_late_return(trans.borrow_date, trans.return_date)

            status = "✅" if not late_info['is_late'] else f"⚠️"

            table_data.append([
                trans.id,
                book.title[:35] if book else "?",
                trans.borrow_date.strftime("%d-%m-%Y"),
                trans.return_date.strftime("%d-%m-%Y"),
                f"{duration['days']}d",
                status
            ])

        print(tabulate(table_data,
                      headers=["ID", "Buku", "Tgl Pinjam", "Tgl Kembali", "Durasi", "Status"],
                      tablefmt="grid"))
    elif choice == 3:
        display_late_returns_report(lib)
    else:
        print("❌ Pilihan tidak valid!")

def menu_statistik_pengembalian(lib):
    """Menu untuk melihat statistik pengembalian"""
    print("\n" + "="*70)
    print("📊 STATISTIK PENGEMBALIAN")
    print("="*70)

    all_transactions = lib.borrow_module.transactions
    returned = [t for t in all_transactions if t.status == "returned"]
    borrowed = [t for t in all_transactions if t.status == "borrowed"]

    # Hitung statistik
    total_returns = len(returned)
    late_returns = 0
    total_fines = 0

    for trans in returned:
        late_info = check_late_return(trans.borrow_date, trans.return_date)
        if late_info['is_late']:
            late_returns += 1
            total_fines += late_info['fine']

    on_time_returns = total_returns - late_returns

    print(f"\n📈 Ringkasan:")
    print(f"{'─'*70}")
    print(f"Total Transaksi      : {len(all_transactions)}")
    print(f"Sedang Dipinjam      : {len(borrowed)} buku")
    print(f"Sudah Dikembalikan   : {total_returns} buku")
    print(f"\n{'─'*70}")
    print(f"✅ Tepat Waktu       : {on_time_returns} ({on_time_returns/max(1,total_returns)*100:.1f}%)")
    print(f"⚠️  Terlambat         : {late_returns} ({late_returns/max(1,total_returns)*100:.1f}%)")
    print(f"💰 Total Denda       : Rp {total_fines:,}")
    print(f"{'─'*70}")

    # Rata-rata durasi peminjaman
    if returned:
        total_days = sum([
            calculate_borrow_duration(t.borrow_date, t.return_date)['days']
            for t in returned
        ])
        avg_days = total_days / len(returned)
        print(f"\n⏱️  Rata-rata Durasi Peminjaman: {avg_days:.1f} hari")

# ============================================================================
# MAIN MENU
# ============================================================================

def show_return_menu():
    """Tampilkan menu pengembalian"""
    print("\n" + "="*70)
    print("📥 MENU PENGEMBALIAN BUKU")
    print("="*70)
    print("\n9.  📥 Kembalikan Buku")
    print("10. 📚 Kembalikan Buku (Batch)")
    print("11. 📜 Riwayat Pengembalian")
    print("12. 📊 Statistik Pengembalian")
    print("0.  ◀️  Keluar")
    print("="*70)

def run_return_menu(lib):
    """Jalankan menu pengembalian"""
    print("\n🚀 MENU PENGEMBALIAN AKTIF")

    while True:
        show_return_menu()
        choice = get_input("\n➤ Pilih (0, 9-12): ", int)

        if choice == 0:
            print("\n👋 Keluar dari Menu Pengembalian")
            break
        elif choice == 9:
            menu_kembalikan_buku(lib)
        elif choice == 10:
            menu_kembalikan_batch(lib)
        elif choice == 11:
            menu_riwayat_pengembalian(lib)
        elif choice == 12:
            menu_statistik_pengembalian(lib)
        else:
            print("❌ Pilihan tidak valid!")

        input("\n⏸️  Tekan ENTER untuk melanjutkan...")

# ============================================================================
# AUTO RUN
# ============================================================================

print("\n" + "="*70)
print("✓ Section 4 siap digunakan!")
print("="*70)

# Auto run
print("\n🚀 Menjalankan Menu Pengembalian...\n")
run_return_menu(library)

# Main
